# Differential evolution with sorted population (JAX compiled)

This example demonstrates the specialised differential evolution
algorithm that keeps each candidate's parameters sorted after every
mutation and Newton refinement. The objective and Newton solver
operate on already sorted arrays, so no additional sorting is done
inside those routines.

In [ ]:
import sys
from pathlib import Path
sys.path.insert(0, str(Path('..') / 'src'))
from kl_decomposition import rectangle_rule
from kl_decomposition.kernel_fit import (
    _prepare_jax_funcs,
    _differential_evolution_sorted,
    newton_with_line_search,
)
import numpy as np

x, w = rectangle_rule(0.0, 2.0, 50)
f = lambda t: np.exp(-t)
target = f(x)
obj, grad, hess = _prepare_jax_funcs(x, target, w, compiled=True, newton=False)def newton_fn(p):
    refined, _ = newton_with_line_search(p, obj, grad, hess, max_iter=1, compiled=True, grad_tol=1e-6)
    return refined

params, info = _differential_evolution_sorted(
    obj,
    2,
    max_gen=20,
    pop_size=20,
    n_newton=2,
    grad=grad,
    grad_tol=1e-6,
    verbose=True,
)
a = np.exp(params[:2])
b = np.exp(params[2:])
print('a:', a)
print('b:', b)
print('best_score:', info.best_score)
print('grad_history:', info.grad_history)